In [ ]:
!pip install -q espnet==0.10.6 pyopenjtalk==0.2 pypinyin==0.44.0 parallel_wavegan==0.5.4 gdown==4.4.0 espnet_model_zoo pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.3/962.3 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 KB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 KB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 KB 22.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Gettin

In [ ]:
from espnet2.bin.tts_inference import Text2Speech
from espnet2.utils.types import str_or_none
import time
import torch
from IPython.display import display, Audio
import soundfile as sf
import pydub
import glob
import os
import re
import numpy as np
import kaldiio
from espnet_model_zoo.downloader import ModelDownloader
import pickle

Advanced voice options : VITS-XVECTOR

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [56]:
# text="The processing time due to a solution will come down from 1 week to 12 minutes. In addition, the solution will save 400 hours of human work every month. Also , it can speed up product launches by 1 month . Communality the solution results in approximately 120 million dollars in gain every year."

In [63]:
text="NLTK is the Natural Language Toolkit library in python that is used for Text Pre-processing. It provides easy-to-use text processing libraries for classification , tokenization , stemming , tagging , parsing , and semantic reasoning ,"

In [64]:
# Preprocessing for Acronyms
# def acronym(text):
#   text = re.sub(r'%',' percent ', text)
#   d=text.split(' ')
#   a=[]
#   i=0
#   c=re.findall(r'\b\n|[A-Z]+\b',text)
#   for j in d:
#     if j.isupper()==False and j not in c:
#       a.append(j)
#     elif j in c:
#       e='.'.join(c[i])
#       i=i+1
#       a.append(e)
#     if i==len(c):
#       continue
#   listToStr = ' '.join([str(elem) for elem in a])
#   return listToStr
# acronym(text)


# pickle.dump(acronym,open('Preprocess.pkl','wb'))
acronym = pickle.load(open('/content/drive/MyDrive/TTS Model/Preprocess.pkl','rb'))
Text=acronym(text)

In [65]:
# ####male
# tag = 'kan-bayashi/libritts_xvector_vits'
# vocoder_tag="parallel_wavegan/libritts_parallel_wavegan.v1.long"
# #####female
# tag = 'kan-bayashi/vctk_full_band_multi_spk_vits'
# vocoder_tag="parallel_wavegan/libritts_parallel_wavegan.v1.long"

# i=int(input("please choose the gender of the voice.Type 1 for Male and 2 for Female \n"))

# if(i==1):
#   tag = 'kan-bayashi/libritts_xvector_vits'
#   vocoder_tag="parallel_wavegan/libritts_parallel_wavegan.v1.long"
#   j='Male'
# else:
#   tag = 'kan-bayashi/vctk_full_band_multi_spk_vits'
#   vocoder_tag="parallel_wavegan/libritts_parallel_wavegan.v1.long"
#   j='Female'

# text2speech = Text2Speech.from_pretrained(
#     model_tag=str_or_none(tag),
#     vocoder_tag=str_or_none(vocoder_tag),
#     #device="cuda",
#     # Only for Tacotron 2 & Transformer
#     threshold=0.5,
#     # Only for Tacotron 2
#     minlenratio=0.0,
#     maxlenratio=10.0,
#     use_att_constraint=False,
#     backward_window=1,
#     forward_window=3,
#     # Only for FastSpeech & FastSpeech2 & VITS
#     speed_control_alpha=1,
#     # Only for VITS
#     noise_scale=0.333,
#     noise_scale_dur=0.333,
# )

# # To pickle text2speeach Variable
# pickle.dump(text2speech,open('TTS_model.pkl','wb'))

# # To pickle model_dir Variable 
#d = ModelDownloader()
#pickle.dump(d.download_and_unpack('kan-bayashi/libritts_xvector_vits'),open('TTS_model_dir.pkl','wb'))

In [66]:
## Load text2speech pickle file
text2speech = pickle.load(open('/content/drive/MyDrive/TTS Model/TTS_model.pkl','rb'))

# Load model_dir pickle file
model_dir_pickle = pickle.load(open('/content/drive/MyDrive/TTS Model/TTS_model_dir.pkl','rb')) 
model_dir = os.path.dirname(model_dir_pickle["train_config"])

# X-vector selection
spembs = None
if text2speech.use_spembs:
    xvector_ark = [p for p in glob.glob(f"{model_dir}/../../dump/**/spk_xvector.ark", recursive=True) if "tr" in p][0]
    xvectors = {k: v for k, v in kaldiio.load_ark(xvector_ark)}
    spks = list(xvectors.keys())

    # randomly select speaker
    random_spk_idx = np.random.randint(0, len(spks))
    print('Please select gender of the speaker: 1 for male and 2 for female')
    a=int(input())
    if a==1:
      spk = '7021_79730'        
      spembs = xvectors[spk]
      print(f"selected spk: {spk}")
    else:
      spk = '4992_41797'
      spembs = xvectors[spk]
      print(f"selected spk: {spk}")
   
# Speaker ID selection
sids = None
if text2speech.use_sids:
    spk2sid = glob.glob(f"{model_dir}/../../dump/**/spk2sid", recursive=True)[0]
    with open(spk2sid) as f:
        lines = [line.strip() for line in f.readlines()]
    sid2spk = {int(line.split()[1]): line.split()[0] for line in lines}

    # randomly select speaker
    sids = np.array(np.random.randint(1, len(sid2spk)))
    spk = sid2spk[int(sids)]
    print(f"selected spk: {spk}")

# Reference speech selection for GST(Global Style Tokens )
speech = None
if text2speech.use_speech:
    speech = torch.randn(50000,) * 0.01

# print("Enter text:")
# x=str(input())
with torch.no_grad():
    start = time.time()
    wav = text2speech(Text, speech=speech, spembs=spembs, sids=sids)["wav"]

rtf = (time.time() - start) / (len(wav) / text2speech.fs)
print(f"RTF = {rtf:5f}")

display(Audio(wav.view(-1).cpu().numpy(), rate=text2speech.fs))
# sf.write("speech_"+j+".wav", wav.to('cpu').numpy(), 22050)

# sound = pydub.AudioSegment.from_wav("/content/speech_"+j+".wav")
# sound.export("voice_"+j+".mp4", format="mp4")

Please select gender of the speaker: 1 for male and 2 for female
2
selected spk: 4992_41797
RTF = 1.240544


**MALE**

tag = 'kan-bayashi/libritts_xvector_vits'
vocoder_tag="parallel_wavegan/libritts_parallel_wavegan.v1.long"

text2speech = Text2Speech.from_pretrained(
    model_tag=str_or_none(tag),
    vocoder_tag=str_or_none(vocoder_tag),
    #device="cuda",
    # Only for Tacotron 2 & Transformer
    threshold=0.5,
    # Only for Tacotron 2
    minlenratio=0.0,
    maxlenratio=10.0,
    use_att_constraint=False,
    backward_window=1,
    forward_window=3,
    # Only for FastSpeech & FastSpeech2 & VITS
    speed_control_alpha=1,
    # Only for VITS
    noise_scale=0.333,
    noise_scale_dur=0.333,
)


d = ModelDownloader()
model_dir = os.path.dirname(d.download_and_unpack(tag)["train_config"])

# X-vector selection
spembs = None
if text2speech.use_spembs:
    xvector_ark = [p for p in glob.glob(f"{model_dir}/../../dump/**/spk_xvector.ark", recursive=True) if "tr" in p][0]
    xvectors = {k: v for k, v in kaldiio.load_ark(xvector_ark)}
    spks = list(xvectors.keys())

    # randomly select speaker
    random_spk_idx = np.random.randint(0, len(spks))
    spk = '1320_122617'#spks[random_spk_idx]
    spembs = xvectors[spk]
    print(f"selected spk: {spk}")

# Speaker ID selection
sids = None
if text2speech.use_sids:
    spk2sid = glob.glob(f"{model_dir}/../../dump/**/spk2sid", recursive=True)[0]
    with open(spk2sid) as f:
        lines = [line.strip() for line in f.readlines()]
    sid2spk = {int(line.split()[1]): line.split()[0] for line in lines}

    # randomly select speaker
    sids = np.array(np.random.randint(1, len(sid2spk)))
    spk = sid2spk[int(sids)]
    print(f"selected spk: {spk}")

# Reference speech selection for GST
speech = None
if text2speech.use_speech:
    # you can change here to load your own reference speech
    # e.g.
    # import soundfile as sf
    # speech, fs = sf.read("/path/to/reference.wav")
    # speech = torch.from_numpy(speech).float()
    speech = torch.randn(50000,) * 0.01

i='male'#
print("Enter text:")
x=str(input())
with torch.no_grad():
    start = time.time()
    wav = text2speech(x, speech=speech, spembs=spembs, sids=sids)["wav"]

rtf = (time.time() - start) / (len(wav) / text2speech.fs)
print(f"RTF = {rtf:5f}")

display(Audio(wav.view(-1).cpu().numpy(), rate=text2speech.fs))
sf.write("speech_"+i+".wav", wav.to('cpu').numpy(), 22050)

sound = pydub.AudioSegment.from_wav("/content/speech"+i+".wav")
sound.export("voice_"+i+".mp3", format="mp3")

**FEMALE**

tag = 'kan-bayashi/vctk_full_band_multi_spk_vits'
vocoder_tag="parallel_wavegan/libritts_parallel_wavegan.v1.long"

text2speech = Text2Speech.from_pretrained(
    model_tag=str_or_none(tag),
    vocoder_tag=str_or_none(vocoder_tag),
    #device="cuda",
    # Only for Tacotron 2 & Transformer
    threshold=0.5,
    # Only for Tacotron 2
    minlenratio=0.0,
    maxlenratio=10.0,
    use_att_constraint=False,
    backward_window=1,
    forward_window=3,
    # Only for FastSpeech & FastSpeech2 & VITS
    speed_control_alpha=1,
    # Only for VITS
    noise_scale=0.333,
    noise_scale_dur=0.333,
)


d = ModelDownloader()
model_dir = os.path.dirname(d.download_and_unpack(tag)["train_config"])

# X-vector selection
spembs = None
if text2speech.use_spembs:
    xvector_ark = [p for p in glob.glob(f"{model_dir}/../../dump/**/spk_xvector.ark", recursive=True) if "tr" in p][0]
    xvectors = {k: v for k, v in kaldiio.load_ark(xvector_ark)}
    spks = list(xvectors.keys())

    # randomly select speaker
    random_spk_idx = 16#np.random.randint(0, len(spks))
    spk = 'p276'#spks[random_spk_idx]##########i have set it dont change it
    spembs = xvectors[spk]
    print(f"selected spk: {spk}")

# Speaker ID selection
sids = None
if text2speech.use_sids:
    spk2sid = glob.glob(f"{model_dir}/../../dump/**/spk2sid", recursive=True)[0]
    with open(spk2sid) as f:
        lines = [line.strip() for line in f.readlines()]
    sid2spk = {int(line.split()[1]): line.split()[0] for line in lines}

    # randomly select speaker
    sids = np.array(np.random.randint(1, len(sid2spk)))
    spk = sid2spk[int(sids)]
    print(f"selected spk: {spk}")

# Reference speech selection for GST
speech = None
if text2speech.use_speech:
    # you can change here to load your own reference speech
    # e.g.
    # import soundfile as sf
    # speech, fs = sf.read("/path/to/reference.wav")
    # speech = torch.from_numpy(speech).float()
    speech = torch.randn(50000,) * 0.01

i='female'
print("Enter text:")
x=str(input())
with torch.no_grad():
    start = time.time()
    wav = text2speech(x, speech=speech, spembs=spembs, sids=sids)["wav"]

rtf = (time.time() - start) / (len(wav) / text2speech.fs)
print(f"RTF = {rtf:5f}")

display(Audio(wav.view(-1).cpu().numpy(), rate=text2speech.fs))
sf.write("speech_"+i+".wav", wav.to('cpu').numpy(), 22050)

sound = pydub.AudioSegment.from_wav("/content/speech"+i+".wav")
sound.export("voice_"+i+".mp3", format="mp3")

In [ ]:
#spks.index('7285_72200')
#spks.index('4160_11549')
#spks.index('8797_294123')# male
#2910_131096
#spks.index('1320_122617')#:male
#7021_79730


#p295 f 17
############################
#p293 female:10
#p341 f :14
#p276 f 16*******
#p267*******f